# Fraud Detection in Financial Transactions

**Author**: Data Science Intern Candidate  
**Date**: February 2026  
**Dataset**: PaySim Synthetic Financial Dataset (~6.3M transactions)

---

## 1. Problem Understanding

### What is Fraud Detection?

Fraud detection in financial services means identifying transactions that are **unauthorized, deceptive, or illegal**. In this context, we're looking at mobile money transactions where criminals may:

- Take over customer accounts
- Transfer funds to accounts they control
- Cash out before the fraud is discovered

### Why Does This Matter for Businesses?

| Impact Area | Consequence |
|-------------|-------------|
| **Financial Loss** | Direct monetary loss from fraudulent transactions |
| **Customer Trust** | Victims may leave the platform permanently |
| **Regulatory Risk** | Fines and penalties for inadequate fraud controls |
| **Operational Cost** | Manual investigation of suspected fraud is expensive |

### What Does Success Look Like?

Success is **not** just high accuracy. In fraud detection, we need to:

1. **Catch most fraudulent transactions** (high recall) — missing fraud costs money
2. **Minimize false alarms** (reasonable precision) — blocking legitimate customers hurts business
3. **Provide actionable insights** — understanding *why* transactions are flagged helps improve controls

The business goal is to **reduce fraud losses while maintaining customer experience**.

## 2. Data Overview & Initial Analysis

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, precision_recall_curve)
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries loaded successfully!")

In [ ]:
# Load data with memory-efficient dtypes
dtype_spec = {
    'step': 'int16',
    'type': 'category',
    'amount': 'float32',
    'nameOrig': 'object',
    'oldbalanceOrg': 'float32',
    'newbalanceOrig': 'float32',
    'nameDest': 'object',
    'oldbalanceDest': 'float32',
    'newbalanceDest': 'float32',
    'isFraud': 'int8',
    'isFlaggedFraud': 'int8'
}

df = pd.read_csv('Fraud.csv', dtype=dtype_spec)
print(f"Dataset loaded: {df.shape[0]:,} transactions, {df.shape[1]} features")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

In [ ]:
# Dataset overview
print("=" * 50)
print("DATASET STRUCTURE")
print("=" * 50)
print(f"\nTime span: {df['step'].max()} hours ({df['step'].max() // 24} days)")
print(f"\nTransaction types: {df['type'].unique().tolist()}")
print(f"\nFeatures:\n{df.dtypes}")

In [ ]:
# Class imbalance analysis
fraud_counts = df['isFraud'].value_counts()
fraud_pct = df['isFraud'].value_counts(normalize=True) * 100

print("=" * 50)
print("CLASS DISTRIBUTION (Target Variable: isFraud)")
print("=" * 50)
print(f"\nLegitimate transactions: {fraud_counts[0]:,} ({fraud_pct[0]:.2f}%)")
print(f"Fraudulent transactions: {fraud_counts[1]:,} ({fraud_pct[1]:.2f}%)")
print(f"\nImbalance ratio: 1 fraud per {fraud_counts[0] // fraud_counts[1]:,} legitimate transactions")

In [ ]:
# Visualize class imbalance
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart
colors = ['#2ecc71', '#e74c3c']
axes[0].bar(['Legitimate', 'Fraud'], fraud_counts.values, color=colors)
axes[0].set_ylabel('Number of Transactions')
axes[0].set_title('Transaction Count by Class')
for i, v in enumerate(fraud_counts.values):
    axes[0].text(i, v + 50000, f'{v:,}', ha='center', fontsize=10)

# Pie chart (zoomed to show fraud)
axes[1].pie([fraud_pct[1], fraud_pct[0]], labels=['Fraud', 'Legitimate'], 
            colors=['#e74c3c', '#2ecc71'], autopct='%.2f%%', startangle=90,
            explode=[0.1, 0])
axes[1].set_title('Fraud Percentage (Highly Imbalanced)')

plt.tight_layout()
plt.show()

In [ ]:
# Key observation: Which transaction types have fraud?
fraud_by_type = df.groupby('type').agg(
    total=('isFraud', 'count'),
    fraud_count=('isFraud', 'sum')
).reset_index()
fraud_by_type['fraud_rate'] = (fraud_by_type['fraud_count'] / fraud_by_type['total'] * 100).round(2)

print("=" * 50)
print("FRAUD DISTRIBUTION BY TRANSACTION TYPE")
print("=" * 50)
print(fraud_by_type.to_string(index=False))
print("\n💡 KEY INSIGHT: Fraud ONLY occurs in TRANSFER and CASH_OUT transactions.")

### Initial Observations

1. **Severe class imbalance**: Fraud represents only ~0.13% of all transactions
2. **Fraud is concentrated**: Only TRANSFER and CASH_OUT transactions contain fraud
3. **This matches the fraud pattern**: Criminals transfer money, then cash out

**Implication**: We should focus our analysis on TRANSFER and CASH_OUT transactions, and use techniques like SMOTE to handle imbalance during modeling.

## 3. Data Cleaning & Preparation

In [ ]:
# Check for missing values
print("=" * 50)
print("MISSING VALUES ANALYSIS")
print("=" * 50)
missing = df.isnull().sum()
print(f"\n{missing[missing > 0] if missing.sum() > 0 else 'No missing values found!'}")

# Check for zero balances in destination (expected for merchants)
merchant_dest = df[df['nameDest'].str.startswith('M')]
print(f"\nMerchant destinations: {len(merchant_dest):,} transactions")
print(f"Merchants with zero balances: {(merchant_dest['oldbalanceDest'] == 0).sum():,}")
print("\n📝 Note: Merchant (M) destinations have no balance info by design.")

In [ ]:
# Focus on fraud-relevant transactions only
df_relevant = df[df['type'].isin(['TRANSFER', 'CASH_OUT'])].copy()
print(f"Filtered to TRANSFER and CASH_OUT: {len(df_relevant):,} transactions")
print(f"Fraud cases retained: {df_relevant['isFraud'].sum():,} (100% of all fraud)")

In [ ]:
# Feature engineering
print("=" * 50)
print("FEATURE ENGINEERING")
print("=" * 50)

# 1. Balance change for origin account
df_relevant['orig_balance_change'] = df_relevant['newbalanceOrig'] - df_relevant['oldbalanceOrg']

# 2. Balance change for destination account  
df_relevant['dest_balance_change'] = df_relevant['newbalanceDest'] - df_relevant['oldbalanceDest']

# 3. Error in origin balance (amount doesn't match balance change)
df_relevant['orig_balance_error'] = (df_relevant['oldbalanceOrg'] - df_relevant['amount'] - df_relevant['newbalanceOrig']).abs()

# 4. Error in destination balance
df_relevant['dest_balance_error'] = (df_relevant['oldbalanceDest'] + df_relevant['amount'] - df_relevant['newbalanceDest']).abs()

# 5. Is the origin account being emptied?
df_relevant['is_orig_emptied'] = ((df_relevant['oldbalanceOrg'] > 0) & 
                                   (df_relevant['newbalanceOrig'] == 0)).astype(int)

# 6. Hour of day (for time patterns)
df_relevant['hour'] = df_relevant['step'] % 24

# 7. Transaction type as binary
df_relevant['is_transfer'] = (df_relevant['type'] == 'TRANSFER').astype(int)

print("Created features:")
print("  • orig_balance_change: Change in sender's balance")
print("  • dest_balance_change: Change in receiver's balance")
print("  • orig_balance_error: Discrepancy in sender's balance calculation")
print("  • dest_balance_error: Discrepancy in receiver's balance calculation")
print("  • is_orig_emptied: Flag if account was emptied to zero")
print("  • hour: Hour of day (0-23)")
print("  • is_transfer: Binary flag for TRANSFER type")

In [ ]:
# Outlier analysis (outliers may indicate fraud!)
print("=" * 50)
print("OUTLIER ANALYSIS")
print("=" * 50)

# Compare amount distributions
print("\nAmount statistics by fraud status:")
print(df_relevant.groupby('isFraud')['amount'].describe().T)

In [ ]:
# Visualize amount distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Log-transformed amount distribution
for label, color in [(0, '#2ecc71'), (1, '#e74c3c')]:
    subset = df_relevant[df_relevant['isFraud'] == label]['amount']
    axes[0].hist(np.log10(subset + 1), bins=50, alpha=0.6, 
                 label='Fraud' if label else 'Legitimate', color=color)
axes[0].set_xlabel('Log10(Amount + 1)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Transaction Amount Distribution')
axes[0].legend()

# Boxplot comparison
df_relevant.boxplot(column='amount', by='isFraud', ax=axes[1])
axes[1].set_title('Amount by Fraud Status')
axes[1].set_xlabel('Is Fraud')
axes[1].set_ylabel('Amount')
plt.suptitle('')

plt.tight_layout()
plt.show()

print("\n💡 KEY INSIGHT: Fraudulent transactions tend to have HIGHER amounts.")
print("   This makes sense - criminals want to maximize their take per transaction.")

In [ ]:
# Correlation analysis
print("=" * 50)
print("CORRELATION WITH FRAUD")
print("=" * 50)

numeric_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 
                'newbalanceDest', 'orig_balance_change', 'dest_balance_change',
                'orig_balance_error', 'dest_balance_error', 'is_orig_emptied', 
                'hour', 'is_transfer', 'isFraud']

corr_with_fraud = df_relevant[numeric_cols].corr()['isFraud'].drop('isFraud').sort_values(key=abs, ascending=False)
print("\nCorrelation with isFraud (sorted by absolute value):")
print(corr_with_fraud.to_string())

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 8))
corr_matrix = df_relevant[numeric_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

### Data Cleaning Summary

**What we did:**
1. ✅ No missing values in the dataset
2. ✅ Filtered to fraud-relevant transactions (TRANSFER, CASH_OUT)
3. ✅ Created domain-relevant features (balance errors, account emptying flag)
4. ✅ Retained outliers — they may indicate fraud behavior

**Key findings:**
- `is_orig_emptied` has strong correlation with fraud (accounts being emptied)
- Balance errors may indicate suspicious activity
- Higher transaction amounts correlate with fraud

## 4. Feature Selection & Reasoning

In [ ]:
# Select features for modeling
print("=" * 50)
print("FEATURE SELECTION")
print("=" * 50)

# Features to use
features = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 
            'newbalanceDest', 'orig_balance_error', 'dest_balance_error',
            'is_orig_emptied', 'hour', 'is_transfer']

print("\nSelected features and rationale:")
print("-" * 50)
feature_rationale = {
    'amount': 'Fraud tends to involve larger amounts',
    'oldbalanceOrg': 'Fraudsters often target accounts with funds',
    'newbalanceOrig': 'Fraud often empties accounts completely',
    'oldbalanceDest': 'Destination patterns may differ for fraud',
    'newbalanceDest': 'Receiving account behavior after transaction',
    'orig_balance_error': 'Calculation discrepancies may indicate tampering',
    'dest_balance_error': 'Unusual destination balance changes',
    'is_orig_emptied': 'Strong fraud signal - account drained to zero',
    'hour': 'Fraud may occur at specific times',
    'is_transfer': 'Transfer vs cash-out fraud patterns differ'
}

for feat, reason in feature_rationale.items():
    print(f"  • {feat}: {reason}")

print("\n❌ Excluded features:")
print("  • nameOrig, nameDest: Customer IDs - not generalizable")
print("  • step: Raw time step - hour captures cyclical patterns")
print("  • type: Encoded as is_transfer")
print("  • isFlaggedFraud: Would leak information (based on >200K rule)")

In [ ]:
# Prepare final dataset
X = df_relevant[features].copy()
y = df_relevant['isFraud'].copy()

print(f"\nFinal dataset: {X.shape[0]:,} samples, {X.shape[1]} features")
print(f"Target distribution: {y.value_counts().to_dict()}")

## 5. Model Selection & Training

In [ ]:
# Train-test split (stratified to preserve fraud ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("=" * 50)
print("TRAIN-TEST SPLIT")
print("=" * 50)
print(f"\nTraining set: {len(X_train):,} samples")
print(f"  - Legitimate: {(y_train == 0).sum():,}")
print(f"  - Fraud: {(y_train == 1).sum():,}")
print(f"\nTest set: {len(X_test):,} samples")
print(f"  - Legitimate: {(y_test == 0).sum():,}")
print(f"  - Fraud: {(y_test == 1).sum():,}")

In [ ]:
# Handle class imbalance with SMOTE (on training set only!)
print("=" * 50)
print("HANDLING CLASS IMBALANCE WITH SMOTE")
print("=" * 50)

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"\nBefore SMOTE: {len(X_train):,} samples")
print(f"After SMOTE: {len(X_train_balanced):,} samples")
print(f"\nBalanced class distribution: {pd.Series(y_train_balanced).value_counts().to_dict()}")
print("\n📝 Note: SMOTE creates synthetic minority samples to balance classes.")
print("   Applied only to training data to avoid data leakage.")

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)
print("Features scaled using StandardScaler.")

In [ ]:
# Model 1: Logistic Regression (Interpretable Baseline)
print("=" * 50)
print("MODEL 1: LOGISTIC REGRESSION")
print("=" * 50)
print("\nWhy Logistic Regression?")
print("  • Interpretable coefficients show feature importance")
print("  • Fast training even on large datasets")
print("  • Good baseline for comparison")
print("  • Outputs probabilities for risk scoring")

lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train_balanced)
print("\n✅ Logistic Regression trained successfully!")

In [ ]:
# Model 2: Random Forest (Non-linear)
print("=" * 50)
print("MODEL 2: RANDOM FOREST")
print("=" * 50)
print("\nWhy Random Forest?")
print("  • Captures non-linear relationships")
print("  • Robust to outliers (important for fraud)")
print("  • Built-in feature importance")
print("  • Handles imbalanced data well with class_weight")

# Using original training data with class_weight instead of SMOTE
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
print("\n✅ Random Forest trained successfully!")

## 6. Model Evaluation

In [ ]:
# Predictions
lr_pred = lr_model.predict(X_test_scaled)
lr_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

rf_pred = rf_model.predict(X_test)
rf_proba = rf_model.predict_proba(X_test)[:, 1]

In [ ]:
# Evaluation function
def evaluate_model(name, y_true, y_pred, y_proba):
    print(f"\n{'=' * 50}")
    print(f"{name} EVALUATION")
    print('=' * 50)
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Legitimate', 'Fraud']))
    
    roc_auc = roc_auc_score(y_true, y_proba)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    
    return roc_auc

lr_auc = evaluate_model("LOGISTIC REGRESSION", y_test, lr_pred, lr_proba)
rf_auc = evaluate_model("RANDOM FOREST", y_test, rf_pred, rf_proba)

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (name, pred) in zip(axes, [('Logistic Regression', lr_pred), ('Random Forest', rf_pred)]):
    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt=',d', cmap='Blues', ax=ax,
                xticklabels=['Predicted Legitimate', 'Predicted Fraud'],
                yticklabels=['Actual Legitimate', 'Actual Fraud'])
    ax.set_title(f'{name} Confusion Matrix')

plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves
plt.figure(figsize=(10, 6))

for name, proba, auc in [('Logistic Regression', lr_proba, lr_auc), 
                          ('Random Forest', rf_proba, rf_auc)]:
    fpr, tpr, _ = roc_curve(y_test, proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Why Accuracy is Misleading

With 99.87% legitimate transactions, a model that predicts **everything as legitimate** would achieve:
- **Accuracy**: 99.87% (looks great!)
- **Fraud Detection**: 0% (completely useless!)

**Business Cost Analysis:**

| Error Type | What Happens | Business Impact |
|------------|--------------|------------------|
| **False Negative** (Missed Fraud) | Fraud gets through | Direct financial loss |
| **False Positive** (False Alarm) | Legitimate blocked | Customer frustration, churn |

In fraud detection, **False Negatives are usually more costly** because:
- Each missed fraud = direct monetary loss
- Criminals learn which patterns work
- Regulatory penalties for inadequate controls

**This is why we prioritize Recall (catching fraud) while maintaining reasonable Precision (not annoying customers).**

## 7. Key Fraud Indicators

In [ ]:
# Feature importance from Random Forest
print("=" * 50)
print("FEATURE IMPORTANCE (Random Forest)")
print("=" * 50)

importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop fraud predictors:")
print(importance_df.to_string(index=False))

In [ ]:
# Visualize feature importance
plt.figure(figsize=(10, 6))
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(importance_df)))
plt.barh(importance_df['Feature'], importance_df['Importance'], color=colors)
plt.xlabel('Importance Score')
plt.title('Feature Importance for Fraud Detection')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Logistic Regression coefficients (interpretability)
print("=" * 50)
print("LOGISTIC REGRESSION COEFFICIENTS")
print("=" * 50)

coef_df = pd.DataFrame({
    'Feature': features,
    'Coefficient': lr_model.coef_[0]
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nInterpretation: Positive = increases fraud probability")
print(coef_df.to_string(index=False))

### Key Fraud Indicators Explained

Based on our analysis, the **strongest fraud indicators** are:

| Indicator | What It Means | Why It Makes Sense |
|-----------|---------------|--------------------|
| **Account Emptying** | Origin account balance goes to zero | Fraudsters want to take everything |
| **High Transaction Amount** | Larger than typical transactions | Maximize gains per attempt |
| **Balance Discrepancies** | Math doesn't add up | Possible record manipulation |
| **Destination Balance Patterns** | Unusual receiving account behavior | Money mule accounts may show patterns |

**Behavioral Pattern of Fraud:**
1. Fraudster gains access to victim's account
2. Transfers maximum available balance
3. Destination account (often a mule) cashes out quickly

This is exactly what the model learned to detect!

## 8. Business & Prevention Strategy

### Recommended Fraud Prevention Framework

Based on our findings, here's a practical prevention strategy:

#### 1. Real-Time Transaction Monitoring

**High-Risk Triggers** (require additional verification):
- Transaction would empty the account (balance → 0)
- Amount exceeds 90th percentile of customer's history
- First-time transfer to new destination
- Multiple large transfers in short period

#### 2. Risk-Based Authentication

| Risk Score | Amount Threshold | Action |
|------------|------------------|--------|
| Low (0-0.3) | Any | Allow immediately |
| Medium (0.3-0.7) | >$10,000 | SMS/Email confirmation |
| High (0.7-0.9) | Any | Phone verification |
| Very High (>0.9) | Any | Manual review + temporary hold |

#### 3. Adaptive Rules Using Model Output

```
IF model_fraud_probability > 0.8:
    BLOCK transaction
    ALERT fraud team
    NOTIFY customer
ELIF model_fraud_probability > 0.5:
    HOLD for 15 minutes
    REQUEST secondary authentication
ELIF account_being_emptied AND destination_is_new:
    REQUEST confirmation
ELSE:
    ALLOW transaction
```

#### 4. Customer Education
- Alerts for unusual account activity
- Regular reminders about security practices
- Easy fraud reporting mechanisms

## 9. Measuring Success of Prevention

### Key Performance Indicators (KPIs)

| Metric | Formula | Target | Why It Matters |
|--------|---------|--------|----------------|
| **Fraud Rate** | Fraud $ / Total $ | Decrease 50%+ | Primary success metric |
| **Detection Rate** | Caught Fraud / Total Fraud | >90% | Model effectiveness |
| **False Positive Rate** | False Blocks / Total Blocks | <5% | Customer experience |
| **Investigation Efficiency** | True Fraud / Investigated Cases | >30% | Analyst productivity |
| **Mean Time to Detect** | Time from fraud to alert | <1 hour | Response speed |

### Monitoring Framework

**Daily:**
- Transaction volumes by risk category
- Blocked transaction count and reasons
- False positive complaints

**Weekly:**
- Model performance metrics (precision, recall, AUC)
- New fraud pattern identification
- Rule effectiveness review

**Monthly:**
- Total fraud losses vs. baseline
- Customer satisfaction scores
- Model drift analysis
- A/B test results for rule changes

### Validating System Improvements

1. **A/B Testing**: Route 5% of traffic through new rules, compare outcomes
2. **Backtesting**: Apply new model to historical data, measure improvement
3. **Champion-Challenger**: Run new model alongside production model
4. **Feedback Loop**: Analyst decisions feed back into model retraining

## 10. Conclusion & Next Steps

### What We Learned

1. **Fraud is rare but concentrated**: Only 0.13% of transactions are fraudulent, and they only occur in TRANSFER and CASH_OUT types

2. **Clear behavioral patterns exist**: Fraudsters tend to empty accounts completely and transfer larger amounts

3. **Simple models work well**: Random Forest achieved strong ROC-AUC, and Logistic Regression provides interpretable insights

4. **Feature engineering matters**: Domain-relevant features like `is_orig_emptied` and balance errors significantly improve detection

### Model Limitations

| Limitation | Impact | Mitigation |
|------------|--------|------------|
| Synthetic data | May not capture all real fraud patterns | Validate on real data before deployment |
| No temporal features | Can't detect velocity patterns | Add transaction frequency features |
| Static model | Won't adapt to new fraud tactics | Implement regular retraining pipeline |
| No customer history | Missing behavioral baselines | Integrate customer profiling |

### Future Improvements

**With More Time:**
- Add customer transaction history features
- Implement network analysis (related accounts)
- Build velocity-based features (transactions per hour/day)
- Test gradient boosting models (XGBoost, LightGBM)

**For Production:**
- Build real-time scoring API
- Implement model monitoring and drift detection
- Create analyst review dashboard
- Set up automated retraining pipeline

---

**Final Note**: This analysis demonstrates that effective fraud detection requires not just technical modeling skills, but also domain understanding and clear business communication. The goal is not the most complex model, but the most *useful* one.

In [ ]:
# Final summary statistics
print("=" * 60)
print("PROJECT SUMMARY")
print("=" * 60)
print(f"\nDataset: {len(df):,} total transactions")
print(f"Analysis subset: {len(df_relevant):,} (TRANSFER + CASH_OUT)")
print(f"Fraud cases: {y.sum():,} ({y.mean()*100:.2f}%)")
print(f"\nModels trained:")
print(f"  • Logistic Regression - ROC-AUC: {lr_auc:.4f}")
print(f"  • Random Forest - ROC-AUC: {rf_auc:.4f}")
print(f"\nTop fraud indicators: {importance_df['Feature'].head(3).tolist()}")
print("\n✅ Analysis complete!")